## 单继承
在单继承中，super主要用来实现对父类方法的调用

In [2]:
class A:
    def __init__(self):
        self.n = 2
        
    def add(self, m):
        print('self is {0} @A.add'.format(self))
        self.n += m
        
class B(A):
    def __init__(self):
        self.n = 3
    
    def add(self, m):
        print('self is {0} @B.add'.format(self))
        super().add(m)
        self.n += 3

b = B()
b.add(2)
print(b.n)

self is <__main__.B object at 0x0000026759E81548> @B.add
self is <__main__.B object at 0x0000026759E81548> @A.add
8


## 多继承
当我们调用 super() 的时候，实际上是实例化了一个 super 类。这个类包含两个重要信息，方法解析列表（MRO）以及当前所在的类。方法解析过程只考虑当前类后的类。
super() 实际上做了啥呢？简单来说就是：提供一个 MRO 列表以及一个 MRO 中的类 C ， super() 将返回一个从 MRO 列表中 C 之后的类中查找方法的对象。

In [5]:
class C(A):
    def __init__(self):
        self.n = 4

    def add(self, m):
        print('self is {0} @C.add'.format(self))
        super().add(m)
        self.n += 4


class D(B, C):
    def __init__(self):
        self.n = 5

    def add(self, m):
        print('self is {0} @D.add'.format(self))
        super().add(m)
        self.n += 5
        
d = D()
d.add(2)
print(d.n)

self is <__main__.D object at 0x0000026759ED2A88> @D.add
self is <__main__.D object at 0x0000026759ED2A88> @B.add
self is <__main__.D object at 0x0000026759ED2A88> @C.add
self is <__main__.D object at 0x0000026759ED2A88> @A.add
19


## 实现一个super类
类需要包含MRO列表和当前类，并包含一个方法，可以返回包含需要查找方法的类

In [ ]:
from functools import partial

class Super():
    def __init__(self, sub_cls, instance):
        # 假设 sub_cls = B, instance = D()
        # Super(B, self).add(233)
        mro = instance.__class__.mro()
        # mro == [D, B, C, A, object]
        # sub_cls is B
        # 从 mro 中 sub_cls 后面的类中进行查找
        # __mro_tail == [C, A, object]
        self.__mro_tail = mro[mro.index(sub_cls)+1:]
        self.__sub_cls = sub_cls
        self.__instance = instance
        
    def __getattr__(self, name):
        # 从 mro tail 列表的各个类中查找方法
        for cls in self.__mro_tail:
            if not hasattr(cls, name):
                continue

            print('call {}.{}'.format(cls, name))
            # 获取类中定义的方法
            attr = getattr(cls, name)
            # 因为 d = D(); d.add(233)  等价于 D.add(d, 233)
            # 所以返回的函数需要自动填充第一个 self 参数
            return partial(attr, self.__instance)

        raise AttributeError(name)
        